In [31]:
# Setup
import pandas as pd

data = pd.read_csv('../src/cleaned_data.csv')
raw = pd.read_csv('../src/raw_data.csv')
popdense = pd.read_csv('../src/external_data/cleaned_with_popdensity.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../src/external_data/cleaned_with_popdensity.csv'

In [3]:
import plotly.express as px
import scipy.stats as stats

# Pt1: Exploriation

In [55]:
# data.head()
data.nunique()
# data.info()


Unnamed: 0          18547
ID                  18547
City                 1788
PostalCode            913
Region                  3
District               43
Province               11
PropertyType            2
PropertySubType        23
Price                1765
SaleType                4
ConstructionYear      204
BedroomCount           26
LivingArea            681
KitchenType             9
Furnished               2
Fireplace               2
Terrace                 2
TerraceArea           181
Garden                  2
GardenArea           2261
Facades                 8
SwimmingPool            2
Condition               7
EPCScore               12
Latitude                5
Longitude               6
Property url        18547
dtype: int64

In [92]:
data.columns

Index(['Unnamed: 0', 'ID', 'City', 'PostalCode', 'Region', 'District',
       'Province', 'PropertyType', 'PropertySubType', 'Price', 'SaleType',
       'ConstructionYear', 'BedroomCount', 'LivingArea', 'KitchenType',
       'Furnished', 'Fireplace', 'Terrace', 'TerraceArea', 'Garden',
       'GardenArea', 'Facades', 'SwimmingPool', 'Condition', 'EPCScore',
       'Latitude', 'Longitude', 'Property url'],
      dtype='object')

In [79]:
# data['PropertySubType'].unique()
# raw_data['District'].unique()
# data['Condition'].unique()
data['SaleType'].unique()


array(['Residential_Sale', 'Annuity_Monthly_Amount',
       'First_Session_With_Reserve_Price', 'Annuity_Without_Lump_Sum'],
      dtype=object)

In [ ]:
# data['Condition'].value_counts()
# data['SaleType'].value_counts()
data['ConstructionYear'].value_counts()

In [83]:
data['ConstructionYear'].mean()

1517.7784008195395

# Pt 2: Exploring stats

How many observations and features do you have?


In [63]:
data.shape

(18547, 28)

What is the proportion of missing values per column? -> double check for bias in proportion: poolspace will be missing for most appartments I guess, change to None instead of missing


In [64]:
# Proportion = n / total (18598 or data.shape[0]) *100
for column in data.columns:
    missingprop = data[column].isna().sum() / data.shape[0] *100
    if missingprop > 0 :
        print(f'{column}: {round(missingprop, 2)}%')

What variables are most subject to outliers?


In [30]:
# calculate IQR for column
for column in data.columns:
    if column in ['Price', 'ConstructionYear','BedroomCount', 'LivingArea', 'TerraceArea', 'GardenArea', 'Facades',]:
        print(column)
        # setting IQR
        data[column].dropna()
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        # identify outliers
        threshold = 1.5
        outliers = data[(data[column] < Q1 - threshold * IQR) | (data[column] > Q3 + threshold * IQR)]
        lower = data[(data[column] < Q1 - threshold * IQR)]
        upper = data[(data[column] > Q3 + threshold * IQR)]
        print(len(outliers), f'outliers\nbelow Q1 - 1.5*IQR: {len(lower)}\nabove Q3 + 1.5*IQR: {len(upper)}')
        # if column == 'ConstructionYear':
        #     print(lower[column])
        #     print(upper[column])

Price
5271 outliers
below Q1 - 1.5*IQR: 0
above Q3 + 1.5*IQR: 5271
ConstructionYear
807 outliers
below Q1 - 1.5*IQR: 807
above Q3 + 1.5*IQR: 0
BedroomCount
1091 outliers
below Q1 - 1.5*IQR: 0
above Q3 + 1.5*IQR: 1091
LivingArea
3889 outliers
below Q1 - 1.5*IQR: 0
above Q3 + 1.5*IQR: 3889
TerraceArea
8524 outliers
below Q1 - 1.5*IQR: 0
above Q3 + 1.5*IQR: 8524
GardenArea
6888 outliers
below Q1 - 1.5*IQR: 0
above Q3 + 1.5*IQR: 6888
Facades
19 outliers
below Q1 - 1.5*IQR: 0
above Q3 + 1.5*IQR: 19


In [ ]:
for column in raw.columns:
    if column in ['Price', 'ConstructionYear','BedroomCount', 'LivingArea', 'TerraceArea', 'GardenArea', 'Facades',]:
        print(column)
        # setting IQR
        raw[column].dropna()
        Q1 = raw[column].quantile(0.25)
        Q3 = raw[column].quantile(0.75)
        IQR = Q3 - Q1
        # identify outliers
        threshold = 1.5
        outliers = raw[(raw[column] < Q1 - threshold * IQR) | (raw[column] > Q3 + threshold * IQR)]
        lower = raw[(raw[column] < Q1 - threshold * IQR)]
        upper = raw[(raw[column] > Q3 + threshold * IQR)]
        # print(len(outliers), f'outliers \nbelow Q1 - 1.5*IQR: {len(lower)}\nabove Q3 + 1.5*IQR: {len(upper)}')
        if column == 'ConstructionYear':
            print(lower[column])
            print(upper[column])



What is the correlation between the variables and the price? Why do you think some variables are more correlated than others?


In [116]:
# pandas get_dummies for one-hot encoding
data_encoded = pd.get_dummies(data, columns=['City', 'Region', 'District', 'Province', 'PropertyType', 'PropertySubType', 'SaleType', 'KitchenType', 'Condition', 'EPCScore'])
# Now, calculate the correlation of these one-hot encoded variables with Price
print(data_encoded.describe())
# correlations = data_encoded.corrwith(data_encoded['Price']).sort_values(ascending=False)
# print(correlations)

         Unnamed: 0            ID    PostalCode         Price  \
count  18547.000000  1.854700e+04  18547.000000  1.854700e+04   
mean    9293.025718  1.098044e+07   4980.949318  5.029269e+05   
std     5367.315589  2.351815e+05   3235.516327  5.782303e+05   
min        0.000000  7.710166e+06   1000.000000  9.980000e+02   
25%     4641.500000  1.092304e+07   1731.000000  2.450000e+05   
50%     9302.000000  1.105838e+07   4300.000000  3.490000e+05   
75%    13940.500000  1.112190e+07   8400.000000  5.350000e+05   
max    18595.000000  1.116037e+07   9991.000000  1.145000e+07   

       ConstructionYear  BedroomCount    LivingArea     Furnished  \
count      18547.000000  18547.000000  18547.000000  18547.000000   
mean        1517.778401      2.926781    168.033105      0.028953   
std          840.534786      1.628605    165.412808      0.167680   
min           -1.000000      0.000000     -1.000000      0.000000   
25%         1874.000000      2.000000     90.000000      0.000000   


In [ ]:

print('Correlations to price:')
for column in data.columns:
    if column in ['ConstructionYear', 'BedroomCount', 'LivingArea', 'Furnished', 'Fireplace', 'Terrace', 'TerraceArea', 'Garden',
       'GardenArea', 'Facades', 'SwimmingPool']:
        print(column, round(data[column].corr(data['Price']), 2))




How are the variables themselves correlated to each other? Can you find groups of variables that are correlated together?


In [97]:
to_cor = data[['Price', 'ConstructionYear', 'BedroomCount', 'LivingArea', 'Furnished', 'Fireplace', 'Terrace', 'TerraceArea', 'Garden',
       'GardenArea', 'Facades', 'SwimmingPool']]
px.imshow(to_cor.corr(numeric_only= True), 
          zmin= -1, zmax= 1, color_continuous_scale= 'rdbu')
#  Set the text_auto argument to  to display text values for each cell

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': array(['Price', 'ConstructionYear', 'BedroomCount', 'LivingArea', 'Furnished',
                          'Fireplace', 'Terrace', 'TerraceArea', 'Garden', 'GardenArea',
                          'Facades', 'SwimmingPool'], dtype=object),
              'xaxis': 'x',
              'y': array(['Price', 'ConstructionYear', 'BedroomCount', 'LivingArea', 'Furnished',
                          'Fireplace', 'Terrace', 'TerraceArea', 'Garden', 'GardenArea',
                          'Facades', 'SwimmingPool'], dtype=object),
              'yaxis': 'y',
              'z': array([[ 1.00000000e+00,  1.09715483e-01,  3.76979095e-01,  4.77392115e-01,
                            4.35663036e-02, -4.48361116e-02,  7.21962138e-02,  1.35018213e-01,
                            3.91582544e-02,  1.28129872e-01,  1.19703088e-01,  2.66292340e-01],
                          [ 1.09715483e-01,  1.00000000e+00, -2.17107886e-02, -7.59764417e-03,
                            1.80688267e-02, -1.13023226e-01,  8.56338316e-02,  1.69486698e-02,
                           -1.51485888e-02, -9.54871839e-03,  2.29845679e-02,  5.53484108e-04],
                          [ 3.76979095e-01, -2.17107886e-02,  1.00000000e+00,  5.66722942e-01,
                           -3.29108050e-02,  4.96678315e-02,  4.37287090e-02,  9.63994106e-02,
                            2.48269248e-01,  1.12805490e-01,  2.27006707e-01,  1.37158732e-01],
                          [ 4.77392115e-01, -7.59764417e-03,  5.66722942e-01,  1.00000000e+00,
                           -3.48984412e-02,  6.45762206e-02,  2.40216558e-02,  8.59634953e-02,
                            1.53503657e-01,  1.45807556e-01,  1.95004418e-01,  2.05631395e-01],
                          [ 4.35663036e-02,  1.80688267e-02, -3.29108050e-02, -3.48984412e-02,
                            1.00000000e+00, -9.44784887e-03,  6.54247567e-03,  8.34522452e-04,
                           -4.05277635e-02, -2.18551313e-03,  1.39777505e-02,  6.43596238e-03],
                          [-4.48361116e-02, -1.13023226e-01,  4.96678315e-02,  6.45762206e-02,
                           -9.44784887e-03,  1.00000000e+00,  5.67036933e-03,  7.80246151e-04,
                            1.55413905e-01,  1.77683091e-02,  1.41025088e-01,  1.43730214e-02],
                          [ 7.21962138e-02,  8.56338316e-02,  4.37287090e-02,  2.40216558e-02,
                            6.54247567e-03,  5.67036933e-03,  1.00000000e+00,  2.09813708e-01,
                            1.50574624e-01, -9.48740555e-03,  2.28367464e-01,  8.42843503e-02],
                          [ 1.35018213e-01,  1.69486698e-02,  9.63994106e-02,  8.59634953e-02,
                            8.34522452e-04,  7.80246151e-04,  2.09813708e-01,  1.00000000e+00,
                            1.39975455e-01,  1.87348729e-02,  9.69762120e-02,  8.53465369e-02],
                          [ 3.91582544e-02, -1.51485888e-02,  2.48269248e-01,  1.53503657e-01,
                           -4.05277635e-02,  1.55413905e-01,  1.50574624e-01,  1.39975455e-01,
                            1.00000000e+00,  3.95439886e-02,  2.74715971e-01,  8.30589331e-02],
                          [ 1.28129872e-01, -9.54871839e-03,  1.12805490e-01,  1.45807556e-01,
                           -2.18551313e-03,  1.77683091e-02, -9.48740555e-03,  1.87348729e-02,
                            3.95439886e-02,  1.00000000e+00,  6.76020794e-02,  8.61886197e-02],
                          [ 1.19703088e-01,  2.29845679e-02,  2.27006707e-01,  1.95004418e-01,
                            1.39777505e-02,  1.41025088e-01,  2.28367464e-01,  9.69762120e-02,
                            2.74715971e-01,  6.76020794e-02,  1.00000000e+00,  1.54515752e-01],
                          [ 2.66292340e-01,  5.53484108e-04,  1.37158732e-01,  2.05631395e-01,

How are the number of properties distributed according to their surface?


Which five variables do you consider the most important and why? -> strongest correlation to price (+ location?)


What are the least/most expensive municipalities in Belgium/Wallonia/Flanders? (in terms of price per m², average price, and median price)

-Add columns (like m²-price)


- correlation matrix


- price by region
